#Deep Mean Shift Clustering

## Reqs

### Download

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
!pip install pillow==4.1.1
# %reload_ext autoreload
# %autoreload
!pip install python-mnist
# !/usr/local/bin/python -m pip install visdom
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip


### Import

In [0]:
import mnist
# import visdom
# vis = visdom.Visdom(port='6006')
import pickle
import torch
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable, Function, gradcheck

## Dataset

### Download

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
root = '/content'
train = torchvision.datasets.MNIST(root, train=True, transform=trans, target_transform=None, download=True)
root = '/content'
test = torchvision.datasets.MNIST(root, train=False, transform=trans, target_transform=None, download=True)

### Load

In [0]:
train_batch_size = 256
test_batch_size = 100
train_loader = torch.utils.data.DataLoader(
                 dataset=train,
                 batch_size=train_batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test,
                batch_size=test_batch_size,
                shuffle=False)

print ('total trainning batch number: '+ str(len(train_loader)))
print ('total testing batch number: '+ str(len(test_loader)))

# AE Def

## Plotter

In [0]:
def plot_regen(model):
  im_data = mnist.MNIST('/content/raw/')

  train_image, train_label = im_data.load_training()

  train_image = np.array(train_image)

  train_label = np.array(train_label)

  train_image = train_image.reshape(-1, 28, 28, 1)

  fig=plt.figure(figsize=(10,40))

  # print(train_image[0].shape)
  num_im = 4
  for i in range(num_im):
    data = np.array(train_image[i], dtype='float')
    data = data.reshape((28, 28))
    fig.add_subplot(1, num_im, i+1)
    plt.imshow(data, cmap='gray')
  plt.show()

  fig=plt.figure(figsize=(10,40))

  # print(train_image[0].shape)
  num_im = 4
  for i in range(num_im):

    data = np.array(train_image[i], dtype='float')
    data = data.reshape((28, 28, 1))
    data = np.transpose(data)
    data = data.reshape((1, 1, 28, 28))

    data = torch.from_numpy(data)
    data = (data.float() - 0.5) / 1.0
    out, embed = model(data.cuda())
#     out = model(data.cuda())
    out = out.cpu().detach()
    out = out.reshape((28, 28))
    out = np.array(out, dtype='float')
    out = (out * 1.0) + 0.5
    out = (out - np.min(out)) * 255 / (np.max(out) - np.min(out))
#     up = np.where(out >= 128)
#     down = np.where(out < 128)
#     out[up] = 1.
#     out[down] = 0.
    fig.add_subplot(1, num_im, i+1)
    plt.imshow(out.T, cmap='gray')
  plt.show()

# plot_regen(model)

## Mean Shift Cluster

In [0]:
class Mean_Shift_Cluster(torch.nn.Module):

  def __init__(self, delta, eta, ms_iter):
    super(Mean_Shift_Cluster, self).__init__()
    self.delta = delta
    self.eta = eta
    self.ms_iter = ms_iter

  def mean_shift_once(self, X):
    S = torch.mm(X.t(), X)
    K = torch.exp(self.delta * S)
    N = list(X.size())[1]
    d = torch.mm(K.t(), torch.ones(N, 1).cuda())
    q = 1 / d
    D_inv = torch.diagflat(q)
    eye = torch.eye(N).cuda()
    P = ((1-self.eta) * eye) + (self.eta * torch.mm(K, D_inv))
    return torch.mm(X, P)
  
  def forward(self, X):
    
    clust_embs = [0] * self.ms_iter
  
    clust_embs[0] = self.mean_shift_once(X)
  
    for it in range(1, self.ms_iter):
      clust_embs[it] = self.mean_shift_once(clust_embs[it-1])
    
    return clust_embs

## Loss

### MS Loss

In [0]:
class Loss(torch.nn.Module):

  def __init__(self, alpha):
    super(Loss, self).__init__()
    self.alpha = alpha

  def cluster_loss(self, embeddings, y):

    num_classes = len(np.unique(y)) 
    num_samples_for_each_class = [0 for i in range(num_classes)] 
    for i in y:
      num_samples_for_each_class[int(i)] += 1

    total = len(y)
    loss = 0
    cos_sim = torch.nn.CosineSimilarity(dim=0)
    
    for i in range(total):
      for j in range(i+1, total):
        w_i = 1 / num_samples_for_each_class[y[i]] 
        w_j = 1 / num_samples_for_each_class[y[j]]
        if y[i] == y[j]:
          loss += (w_i * w_j) * (1 - cos_sim(embeddings[i] , embeddings[j])) 
        else:
          loss += (w_i * w_j) * (torch.clamp(cos_sim(embeddings[i] , embeddings[j]) - alpha , 0)) 

    return loss/y.shape[0]
  
  def regen_loss(self, X, X_):
    
    ae_loss = torch.nn.MSELoss()
    regen_loss = ae_loss(X, X_)/(X.size()[0]*X.size()[1]*X.size()[2]*X.size()[3])

    return regen_loss
    
  def forward(self, X, X_, embeddings, y):
    
    ms_loss = self.cluster_loss(embeddings[0].t(), y)/len(embeddings)
    
    for i in range(1, len(embeddings)):
      ms_loss += self.cluster_loss(embeddings[i].t(), y)/len(embeddings)
    
    ae_loss = self.regen_loss(X, X_)

    return ms_loss + ae_loss, ms_loss.cpu().detach().numpy(), ae_loss.cpu().detach().numpy()


## Weigh init

In [0]:
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
    nn.init.xavier_uniform_(m.weight.data)


## Model

In [0]:
class AE(nn.Module):
  def __init__(self, z_len):
    super(AE, self).__init__()

    self.conv_1 = nn.Conv2d(1, 20, kernel_size=5, stride=1)
    self.pool = nn.MaxPool2d(2, 2, return_indices=True)
    self.ind1 = 0
    self.ind2 = 0
    self.conv_2 = nn.Conv2d(20, 50, kernel_size=5, stride=1)
    self.fc_1 = nn.Linear(4*4*50, 500)
    self.fc_2 = nn.Linear(500,z_len)
    self.relu = nn.ReLU()

#     self.alpha = alpha
#     self.delta = 3 / (1-self.alpha)
#     self.eta = eta
#     self.ms_iter = ms_iter

    self.upfc_1 = nn.Linear(z_len, 500)
    self.upfc_2 = nn.Linear(500, 4*4*50)
    self.unpool = nn.MaxUnpool2d(2, 2)
    self.upconv_1 = nn.ConvTranspose2d(50, 20, kernel_size=5, stride=1)
    self.upconv_2 = nn.ConvTranspose2d(20, 1, kernel_size=5, stride=1)  

  def encode(self, x):
    z = self.conv_1(x)
    z = self.relu(z)
    z, self.ind1 = self.pool(z)
    z = self.conv_2(z)
    z = self.relu(z)
    z, self.ind2 = self.pool(z)
    z = z.view(z.size(0), -1)
    z = self.fc_1(z)
    z = self.relu(z)
    z = self.fc_2(z)
    z_norm = z.norm(p=2, dim=1, keepdim=True).detach()
    z = z.div(z_norm.expand_as(z))
    return z

#   def mean_shift_cluster(self, X, delta, eta):
#     S = torch.mm(X.t(), X)
#     K = torch.exp(delta * S)
#     N = list(X.size())[1]
#     d = torch.mm(K.t(), torch.ones(N, 1).cuda())
#     q = 1 / d
#     D_inv = torch.diagflat(q)
#     eye = torch.eye(N).cuda()
#     P = ((1-eta) * eye) + (eta * torch.mm(K, D_inv))
#     return torch.mm(X, P)

  def decode(self, z):
    x = self.upfc_1(z)
    x = self.relu(x)
    x = self.upfc_2(x)
    x = x.view(x.size(0), 50, 4, 4)
    x = self.unpool(x, self.ind2)
    x = self.relu(x)
    x = self.upconv_1(x)
    x = self.unpool(x, self.ind1)
    x = self.relu(x)
    x = self.upconv_2(x)
    x = (x - 0.5) / 1.0
    return x

  def forward(self, x):
    
    z = self.encode(x)

#     embeds = []
#     for i in range(self.ms_iter):
#       embeds.append(self.mean_shift_cluster(z.t(), self.delta, self.eta))

#     x = self.decode(embeds[-1].t())
  
    x = self.decode(z)
    
    return x, z.t()
#     return x, embeds

# Model Init

## Params

In [0]:
alpha = 0.5 
eta = 1 
ms_iter = 3
delta = 3 / (1-alpha)
total_epochs = 60
embedding_dim = 64

## Init

In [0]:
model = AE(embedding_dim)
model.apply(weights_init)
model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)

ms_clust = Mean_Shift_Cluster(delta, eta, ms_iter)

ms_ae_loss = Loss(alpha)

# Train

In [0]:
# model = AE(embedding_dim)
# model.load_state_dict(torch.load('/gdrive/My Drive/mean shift data/ms_ae_final.pt'))
# model = model.cuda()

epoch_avg_total_loss = [0]*total_epochs

epoch_avg_cluster_loss = [0]*total_epochs

epoch_avg_regen_loss = [0]*total_epochs

# total_loss_win = vis.line(
#     Y=np.zeros((1)),
#     X=np.zeros((1)),
#     opts=dict(xlabel='epoch',title='Total Loss',ylabel='training loss',legend=['Loss']))

# cluster_loss_win = vis.line(
#     Y=np.zeros((1)),
#     X=np.zeros((1)),
#     opts=dict(xlabel='epoch',title='Cluster Loss',ylabel='training loss',legend=['Loss']))

# regen_loss_win = vis.line(
#     Y=np.zeros((1)),
#     X=np.zeros((1)),
#     opts=dict(xlabel='epoch',title='Regen Loss',ylabel='training loss',legend=['Loss']))

for epoch in range(total_epochs):
  
  print('epoch', epoch)
  
  avg_total_loss = 0
  avg_cluster_loss = 0
  avg_regen_loss = 0
    
  for batch_idx, (x, target) in enumerate(train_loader):

    optimizer.zero_grad()        
    x, target = x.cuda(), target.numpy()

    x_, z = model(x)
    
    clust_embs = ms_clust(z)
    
    total_loss, cluster_loss, regen_loss = ms_ae_loss(x, x_, clust_embs, target)
    
    print('epoch', epoch, 'batch', batch_idx, 'batch total loss', total_loss.cpu().detach().numpy())
    print('epoch', epoch, 'batch', batch_idx, 'batch cluster loss', cluster_loss)
    print('epoch', epoch, 'batch', batch_idx, 'batch regen loss', regen_loss)
    
    total_loss.backward()
    optimizer.step()
    
    avg_total_loss += total_loss.cpu().detach().numpy()
    avg_cluster_loss += cluster_loss
    avg_regen_loss += regen_loss
    
    if batch_idx % 50 == 0:
      print('saving...')
      torch.save(model.state_dict(), '/gdrive/My Drive/mean shift data/ms_ae_final.pt')
      print()
   
  epoch_avg_total_loss[epoch] = avg_total_loss/total_epochs
  epoch_avg_cluster_loss[epoch] = avg_cluster_loss/total_epochs
  epoch_avg_regen_loss[epoch] = avg_regen_loss/total_epochs
  
  loss_file = open('/gdrive/My Drive/mean shift data/loss_list_final.pkl', 'wb')
  pickle.dump([epoch_avg_total_loss, epoch_avg_cluster_loss, epoch_avg_regen_loss], loss_file)
  # pickle.dump([epoch_avg_total_loss], loss_file)
  loss_file.close()

  print('epoch', epoch, 'epoch avg cluster loss', epoch_avg_total_loss[epoch])
  print('epoch', epoch, 'epoch avg cluster loss', epoch_avg_cluster_loss[epoch])
  print('epoch', epoch, 'epoch avg regen loss', epoch_avg_regen_loss[epoch])
  print()
#   vis.line(X=np.ones(1)*(epoch+1),Y=np.array([epoch_avg_total_loss[epoch]]),win=total_loss_win,update='append')
#   vis.line(X=np.ones(1)*(epoch+1),Y=np.array([epoch_avg_cluster_loss[epoch]]),win=cluster_loss_win,update='append')
#   vis.line(X=np.ones(1)*(epoch+1),Y=np.array([epoch_avg_regen_loss[epoch]]),win=regen_loss_win,update='append')
  plot_regen(model)
  print('saving...')
  torch.save(model.state_dict(), '/gdrive/My Drive/mean shift data/ms_ae_final.pt')
  print()
 
  

# Scratch Code

In [0]:
X = torch.randn((100, 1, 28, 28))

model = AE().cuda()

X_ = model(X)
print(X_.size())

In [0]:
def plot_regen(model):
  im_data = mnist.MNIST('/content/raw/')

  train_image, train_label = im_data.load_training()

  train_image = np.array(train_image)

  train_label = np.array(train_label)

  train_image = train_image.reshape(-1, 28, 28, 1)

  fig=plt.figure(figsize=(17,70))

  # print(train_image[0].shape)
  num_im = 4
  for i in range(num_im):
    data = np.array(train_image[i], dtype='float')
    data = data.reshape((28, 28))
    fig.add_subplot(1, num_im, i+1)
    plt.imshow(data, cmap='gray')
  plt.show()

  fig=plt.figure(figsize=(17,70))

  # print(train_image[0].shape)
  num_im = 4
  for i in range(num_im):

    data = np.array(train_image[i], dtype='float')
    data = data.reshape((28, 28, 1))
    data = np.transpose(data)
    data = data.reshape((1, 1, 28, 28))

    data = torch.from_numpy(data)
    data = (data.float() - 0.5) / 1.0
    out, embed = model(data.cuda())

    out = out.cpu().detach()
    out = out.reshape((28, 28))
    out = np.array(out, dtype='float')
    out = (out * 1.0) + 0.5
    out = (out - np.min(out)) * 255 / (np.max(out) - np.min(out))
    up = np.where(out >= 128)
    down = np.where(out < 128)
    out[up] = 1.
    out[down] = 0.
    fig.add_subplot(1, num_im, i+1)
    plt.imshow(out.T, cmap='gray')
  plt.show()
  
plot_regen(model)

In [0]:
from google.colab import drive

drive.mount('/gdrive')

# torch.save(the_model.state_dict(), '/con')

In [0]:
torch.save(model.state_dict(), '/gdrive/My Drive/mean shift data/ms_ae.pt')

In [0]:
model2 = AE()
model2.load_state_dict(torch.load('/gdrive/My Drive/mean shift data/ms_ae.pt'))

In [0]:
model2 = model2.cuda()
data = np.array(train_image[7], dtype='float')
data = data.reshape((28, 28, 1))
data = np.transpose(data)
data = data.reshape((1, 1, 28, 28))

data = torch.from_numpy(data)
# print(data.size())
data = (data.float() - 0.5) / 1.0
out, embed = model2(data.cuda())

out = out.cpu().detach()
out = out.reshape((28, 28))
out = np.array(out, dtype='float')
# print(out)
out = (out * 1.0) + 0.5
out = (out - np.min(out)) * 255 / (np.max(out) - np.min(out))
plt.imshow(out.T, cmap='gray')
plt.show()


In [0]:
file = open('/gdrive/My Drive/mean shift data/loss_list_3.pkl', 'rb')
temp = pickle.load(file)
loss = temp[0]
# file = open('/gdrive/My Drive/mean shift data/loss_list_1.pkl', 'rb')
# temp = pickle.load(file)
# loss = np.concatenate((loss, temp[0]))
# file = open('/gdrive/My Drive/mean shift data/loss_list_2.pkl', 'rb')
# temp = pickle.load(file)
# loss = np.concatenate((loss, temp[0]))

In [0]:
plt.plot(loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('autoencoder w/o mean shift')
plt.show()

In [0]:
def mean_shift_once(X, delta, eta=1):
  
  K = torch.exp(delta * torch.mm(X.t(), X))
  
  N = list(X.size())[1]
  ones = torch.ones(N, 1).cuda()
  D = torch.diagflat(torch.mm(K.t(), ones))
  D_inv = D.inverse()
  
  eye = torch.eye(N).cuda()
  
  shift = (eta * torch.mm(K, D_inv)) + ((1 - eta) * eye)
  X = torch.mm(X, shift)
  
  return X

x = torch.randn((60000, 10)).cuda()
out = mean_shift_once(x, 1/(1+0.35))

x.size()

In [0]:
def cluster_loss(embeddings, y, alpha):
        
        num_classes = len(np.unique(y)) 
        num_samples_for_each_class = [0 for i in range(num_classes)] 
        for i in y:
            num_samples_for_each_class[int(i)] += 1
        
        total = len(y)
        loss = 0
        cos_sim = torch.nn.CosineSimilarity(dim=0)
        for i in range(total):
            for j in range(i+1, total):
                w_i = 1 / num_samples_for_each_class[y[i]] 
                w_j = 1 / num_samples_for_each_class[y[j]]
                if y[i] == y[j]:
                    loss += (w_i * w_j) * (1 - cos_sim(embeddings[i] , embeddings[j])) 
                else:
                    loss += (w_i * w_j) * (torch.clamp(cos_sim(embeddings[i] , embeddings[j]) - alpha , 0)) 

        return loss/(y.shape[0]*10)

model3 = AE()

model3 = model3.cuda()

for batch_idx, (x, target) in enumerate(train_loader):

  total_loss = 0

  x, target = x.cuda(), target.detach()
  
  out = model3.encode(x)
  
  cluster = mean_shift_once(out, 1/(1+0.35))
  
  loss = cluster_loss(cluster, target, 0.5)
  
  print(loss)
  
  break
  
  

In [0]:
# !/usr/local/bin/python -m pip install visdom
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
port = 6006
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port ' + str(port) + ' >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port ' + str(port) + '>> url.txt 2>&1 &')

get_ipython().system_raw('./ngrok http ' + str(port) + ' &')
print('curl')
! curl -s http://localhost:4040/api/tunnels | python -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


In [0]:

print(vis)
vis.text('testing')